# Lab 15 Amazon DynamoDB

### Step 1: To install the AWS CLI version 1 with pip3 (Windows)

If you're using a Mac Computer, the installation instruction is here https://docs.aws.amazon.com/cli/latest/userguide/install-macos.html

If you use Python version 3+, we recommend that you use the pip3 command.

Open the Command Prompt from the Start menu.

Install the AWS CLI version 1 using pip.

```
pip3 install awscli
```

Verify that the AWS CLI version 1 is installed correctly.

```
aws --version
# aws-cli/1.16.273 Python/3.7.3 Windows/10 botocore/1.13.0
```

if the `aws --version` command fails due to unrecognized `aws` command, please refer to the documentation here on how to set up the path variables
https://docs.aws.amazon.com/cli/latest/userguide/install-windows.html


### Step 2: Create AWS User credential

In order to connect to AWS via boto3, we need a user with appropriate privilege. for the purpose of this lab, we need this user to have full access to s3 and dynamodb.

1. Go to AWS console for IAM (identity and access management): https://console.aws.amazon.com/iam/home#/home

2. Click **`users`** on the left-hand menu, then "**Add user**"

![Add users](iam.jpg)

3. Give the user a name such as "dynamodb_lab", check **"programmatic access"**, then click "**Next: permissions**"

4. At the permission page, check attach existing policies directory, search for **s3**, and check "**AmazonS3FullAccess**", and then search for "dynamodb" and check "**AmazonDynamoDBFullAccess**".

5. Accept the default on the rest pages. In the review page, make sure ou see "AmazonS3FullAccess" and "AmazonDynamoDBFullAccess" policies selected. Then **create user**.

![review policy](review.jpg)

6. Save the **Secret access key** for later use (you will be shown this only once).

### Step 3: Configure and Test AWS CLI

At the command line, enter

```
aws configure
```

It will ask you for:
- aws_access_key_id: enter the access key id you've just created
- aws_secret_access_key: enter the access key secret you have just saved
- region: enter `us-east-1`
- accept default with others: by hit "enter"



### Step 4: test your AWS connection with S3 bucket operations

To use Boto 3, you must first import it and tell it what service you are going to use:

In [ ]:
import boto3

# Let's use Amazon S3
s3 = boto3.resource('s3')

Now that you have an `s3` resource, you can make requests and process responses from the service. The following uses the `buckets` collection to print out all bucket names:

In [ ]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

It's also easy to upload and download binary data. For example, the following uploads a new file to S3. Replace `'msbabigdata'` with one of your own existing bucket.

In [ ]:
# Upload a new file
data = open('aws.png', 'rb')

In [ ]:
mybucket = 'msbabigdata'
s3.Bucket(mybucket).put_object(Key='aws.png', Body=data)

To verify that the file has been uploaded, use the following script:

In [ ]:
for my_object in s3.Bucket(mybucket).objects.filter(Prefix="aws"):
    print(my_object.key)

The delete the uploaded file

In [ ]:
s3.Object(mybucket,'aws.png').delete()

Verify again (expect no result)

In [ ]:
for my_object in s3.Bucket(mybucket).objects.filter(Prefix="aws"):
    print(my_object.key)

### Step 5: Dynamdb Operations

By following this guide, you will learn how to use the [`DynamoDB.ServiceResource`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.ServiceResource.create_table) and [`DynamoDB.Table`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table) resources in order to create tables, write items to tables, modify existing items, retrieve items, and query/filter the items in the table.

The documentation for boto3 can be found [here](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html). 

#### 1. Create a table

In order to create a new table, use the [`DynamoDB.ServiceResource.create_table()`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Client.create_table) method. For example:

```python
import boto3

# Get the service resource.
dynamodb = boto3.resource('dynamodb')

# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName='music',
    KeySchema=[
        {
            'AttributeName': 'Artist',
            'KeyType': 'HASH'   # indicate partition key
        },
        {
            'AttributeName': 'songTitle',
            'KeyType': 'RANGE'  # indicate sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'Artist',
            'AttributeType': 'S'   # other values include 'S'|'N'|'B'
        },
        {
            'AttributeName': 'songTitle',
            'AttributeType': 'S'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='music')

# Print out some data about the table.
print(table.item_count)
# expect: 0
```

Following the above example, create a table of `users` with `username` and `last_name` as partition and sort keys respectively.


#### 2\. Using an Existing Table

It is also possible to create a [DynamoDB.Table](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table) resource from an existing table:



In [ ]:
# Instantiate a table resource object without actually
# creating a DynamoDB table. Note that the attributes of this table
# are lazy-loaded: a request is not made nor are the attribute
# values populated until the attributes
# on the table resource are accessed or its load() method is called.
table = dynamodb.Table('users')

# Print out some data about the table.
# This will cause a request to be made to DynamoDB and its attribute
# values will be set based on the response.
print(table.creation_date_time)

#### 3\. Creating a New Item


Once you have a DynamoDB.Table resource you can add new items to the table using `DynamoDB.Table.put_item()` ([documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Client.put_item)). For example:

```python
table.put_item(
   Item={
        'Artist': 'Celine Dion',
        'songTitle': 'Courage',  
        'writers':['Stephan Moccio','Erik Alcock','Rodrigues'],
        'releaseDate':2019
    }
)
```

Following the above example, add a new user into the `users` table with the following info.

- `username`: janedoe
- `first_name`: Jane
- `last_name`: Doe
- `age`: 25
- `account_type`: standard_user

#### 4. Getting an Item

You can then retrieve the object using [DynamoDB.Table.get_item()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.get_item), such as

```python
response = table.get_item(
    Key={
        'Artist': 'Celine Dion',
        'songTitle': 'Courage'
    }
)
item = response['Item']
item
```

Following the above example, retrive the item that you've just added:

#### 5. Updating Item

You can then update attributes of the item in the table using [update_item(https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Client.update_item) as example shown below:

```
import decimal
table.update_item(
    Key={
        'Artist': 'Celine Dion',
        'songTitle': 'Courage'
    },
    UpdateExpression="set releaseDate = :r, number_in_alubum = :n, album_name=:a",
    ExpressionAttributeValues={
        ':r': '2019-09-18',
        ':n': decimal.Decimal('5'),
        ':a': 'Courage'
    }
)
```

Following the above example, set the "age" of the user to 26 and verify the new item.

Then verify that the item has been updated.

#### 6. Deleting Item
You can also delete the item using [DynamoDB.Table.delete_item()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.delete_item) by providing the `Key` as the only argument. Delete the only item in the table.

#### 7. Batch Writing
If you are loading a lot of data at a time, you can make use of [DynamoDB.Table.batch_writer()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.batch_writer) so you can both speed up the process and reduce the number of write requests made to the service.

This method returns a handle to a batch writer object that will automatically handle buffering and sending items in batches. In addition, the batch writer will also automatically handle any unprocessed items and resend them as needed. All you need to do is call `put_item` for any items you want to add, and delete_item for any items you want to delete:

In [ ]:
with table.batch_writer() as batch:
    batch.put_item(
        Item={
            'account_type': 'standard_user',
            'username': 'johndoe',
            'first_name': 'John',
            'last_name': 'Doe',
            'age': 25,
            'address': {
                'road': '1 Jefferson Street',
                'city': 'Los Angeles',
                'state': 'CA',
                'zipcode': 90001
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'super_user',
            'username': 'janedoering',
            'first_name': 'Jane',
            'last_name': 'Doering',
            'age': 40,
            'address': {
                'road': '2 Washington Avenue',
                'city': 'Seattle',
                'state': 'WA',
                'zipcode': 98109
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'standard_user',
            'username': 'bobsmith',
            'first_name': 'Bob',
            'last_name':  'Smith',
            'age': 18,
            'address': {
                'road': '3 Madison Lane',
                'city': 'Louisville',
                'state': 'KY',
                'zipcode': 40213
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'super_user',
            'username': 'alicedoe',
            'first_name': 'Alice',
            'last_name': 'Doe',
            'age': 27,
            'address': {
                'road': '1 Jefferson Street',
                'city': 'Los Angeles',
                'state': 'CA',
                'zipcode': 90001
            }
        }
    )

The batch writer is even able to handle a very large amount of writes to the table.

#### 8. Querying and Scanning

With the table full of items, you can then query or scan the items in the table using the [DynamoDB.Table.query()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.query) or [DynamoDB.Table.scan()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.scan) methods respectively. To add conditions to scanning and querying the table, you will need to import the [boto3.dynamodb.conditions.Key](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/customizations/dynamodb.html#boto3.dynamodb.conditions.Key) and [boto3.dynamodb.conditions.Attr](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/customizations/dynamodb.html#boto3.dynamodb.conditions.Attr) classes. The boto3.dynamodb.conditions.Key should be used when the condition is related to **the key of the item**. The boto3.dynamodb.conditions.Attr should be used when the condition is related to **an attribute of the item**. 

the Key object has the following methods:
- begins_with(value)
- between(low, high)
- eq(value)
- gt(value), lt(value)
- gte(value), lte(value)


the Attr object has the above methods, plus:
- contains(value)
- exists(): whether the attribute exists
- not_exists(): whether the attribute exists
- is_in(value_list)
- size()
- ne(): not equal to a value

For example: 

```python
from boto3.dynamodb.conditions import Key, Attr

response = table.query(
    KeyConditionExpression=Key('Artist').gt('B')
)

response['Items']
```

Following the guidelines in the above, find the item whose username equals "johndoe".

Similarly you can scan the table based on attributes of the items. For example, this scans for all the songs released in 2019:

```python
response = table.scan(
    FilterExpression=Attr('releaseDate').contains('2019')
)
response["Items"]
```

Find all users whose age  is less than 27:

You are also able to chain conditions together using the logical operators: `&` (and), `|` (or), and `~` (not). For example, this scans for all users whose `first_name` starts with `J` and whose `account_type` is `super_user`:

In [ ]:
response = table.scan(
    FilterExpression=Attr('first_name').begins_with('J') & Attr('account_type').eq('super_user')
)
response['Items']

#### 9. Deleting a Table

Finally, if you want to delete your table call [DynamoDB.Table.delete()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.delete).

Delete the table that you created (so that it will start to accumulate cost to your account).

#### 11. Verify that the table has been deleted.

Go to the [AWS console for DynamoDB](https://console.aws.amazon.com/dynamodb/home?region=us-east-1#tables:), to make sure that you don't have any table left.
- note that we have asked you to create tables in us-ease-1 region (in AWS configuration). make sure you have checked the right region.